In [104]:
import pandas as pd

data = pd.read_csv('1612340_19800101_20221231.csv')
st = data['datetime'].iloc[0]
st
data['datetime'] = pd.to_datetime(data['datetime'])
data.index = pd.DatetimeIndex(data.datetime)
data = data.dropna()
#data.plot()

data['datetime'].isnull().values.any()

False

In [64]:
SL = data['hourly_height_STND_meters']
SL

# remove missing values
#SL = data[~SL.isna()]

missing = data[SL.isna()]
missing
#missing.to_csv('missing_values.csv', index=False)

,datetime,hourly_height_STND_meters


In [106]:
# convert datetimes to type usuable for linear regression
t = data['datetime']
# convert to date delta (https://stackoverflow.com/questions/24588437/convert-date-to-float-for-linear-regression-on-pandas-data-frame)
dd = (t - t.min())  / np.timedelta64(1,'D')
dd

datetime
1980-01-01 00:00:00        0.000000
1980-01-01 01:00:00        0.041667
1980-01-01 02:00:00        0.083333
1980-01-01 03:00:00        0.125000
1980-01-01 04:00:00        0.166667
                           ...     
2022-12-31 19:00:00    15705.791667
2022-12-31 20:00:00    15705.833333
2022-12-31 21:00:00    15705.875000
2022-12-31 22:00:00    15705.916667
2022-12-31 23:00:00    15705.958333
Name: datetime, Length: 375128, dtype: float64

In [101]:
from scipy import signal

detrended = signal.detrend(SL)

detrended_df = pd.DataFrame(detrended)
detrended_df

,0
0,0.106690
1,0.072689
2,0.011689
3,-0.076311
4,-0.143311
...,...
375123,0.016786
375124,0.032786
375125,0.026785
375126,0.007785


Maybe we can just calculate it like this:

In [115]:
data.resample('D').max()

,datetime,hourly_height_STND_meters
datetime,,
1980-01-01,1980-01-01 23:00:00,1.954
1980-01-02,1980-01-02 23:00:00,1.923
1980-01-03,1980-01-03 23:00:00,1.875
1980-01-04,1980-01-04 23:00:00,1.780
1980-01-05,1980-01-05 23:00:00,1.759
...,...,...
2022-12-27,2022-12-27 23:00:00,1.882
2022-12-28,2022-12-28 23:00:00,1.754
2022-12-29,2022-12-29 23:00:00,1.685


In [99]:
#tt = np.vstack([dd, np.ones(len(dd))]).T
tt = np.column_stack((dd, np.ones(len(dd))))
tt
b = np.linalg.lstsq(tt, SL, rcond=None)[0]
b
slope = b[0] * 365.25 # m/yr
yh = tt * b
yh

array([[0.00000000e+00, 1.39326306e+00],
       [2.24985663e-07, 1.39326306e+00],
       [4.49971327e-07, 1.39326306e+00],
       ...,
       [8.48063209e-02, 1.39326306e+00],
       [8.48065459e-02, 1.39326306e+00],
       [8.48067709e-02, 1.39326306e+00]])

In [98]:
yh = np.array([1, 2, 3, 4, 5])
idx = 2

# Perform element-wise subtraction
yh = yh - yh[idx]

print(yh)

[-2 -1  0  1  2]


In [96]:
idx = data['datetime'].searchsorted(datetime(2000, 7, 1))
yh = yh - yh[idx] # center around midpoint
h = SL - yh

ValueError: operands could not be broadcast together with shapes (375128,) (375128,2) 

In [97]:
from datetime import datetime

# center around 2000
#idx = np.where(t == datetime(2000, 7, 1).toordinal())[0][0] # find midpoint of the year 2000 (July 1, 2000)
idx = data['datetime'].searchsorted(datetime(2000, 7, 1))
yh = yh - yh[idx] # center around midpoint
yh
#h = SL - yh # detrend timeseries

array([[-0.04042722,  0.        ],
       [-0.040427  ,  0.        ],
       [-0.04042677,  0.        ],
       ...,
       [ 0.0443791 ,  0.        ],
       [ 0.04437932,  0.        ],
       [ 0.04437955,  0.        ]])